<a href="https://colab.research.google.com/github/xuxinyue18-dot/deepseekAPI/blob/main/deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="sk-a7d86afd5c5d46f6bcff42afc55fbc96", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! How can I assist you today? 😊


In [ ]:
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import quote

# 初始化 DeepSeek API 客户端
client = OpenAI(api_key="sk-a7d86afd5c5d46f6bcff42afc55fbc96", base_url="https://api.deepseek.com")

def search_web(query, num_results=3):
    """
    使用 Serper API 进行网络搜索
    """
    try:
        headers = {
            "X-API-KEY": "ad64e51d04b48953f7aa8907d28e05db00a2d89b",
            "Content-Type": "application/json"
        }
        payload = {
            "q": query,
            "num": num_results
        }
        response = requests.post(
            "https://google.serper.dev/search",
            headers=headers,
            json=payload
        )

        # **检查 HTTP 状态码**
        if response.status_code != 200:
            return f"搜索API请求失败: HTTP {response.status_code} - {response.text}"

        # **检查响应内容是否为空**
        if not response.text.strip():
            return "搜索API返回空响应"

        # **尝试解析 JSON**
        try:
            return response.json()
        except json.JSONDecodeError:
            return f"搜索API返回格式错误: {response.text}"

    except requests.RequestException as e:
        return f"搜索API请求错误: {str(e)}"

def get_webpage_content(url):
    """
    获取网页内容并提取文本
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)

        # **检查 HTTP 状态码**
        if response.status_code != 200:
            return f"网页请求失败: HTTP {response.status_code}"

        soup = BeautifulSoup(response.text, 'html.parser')

        for script in soup(['script', 'style']):
            script.decompose()

        text = soup.get_text(separator='\n', strip=True)
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        return '\n'.join(lines)[:1000]

    except requests.RequestException as e:
        return f"获取网页内容出错: {str(e)}"

def get_ai_response(messages):
    """
    获取 AI 响应
    """
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"API调用出错: {str(e)}"

# 初始化基础系统提示
SYSTEM_PROMPT = {
    "role": "system",
    "content": "You are a helpful assistant. When users specifically request web search with '搜索：', you'll provide information based on search results. Otherwise, you'll respond based on your own knowledge."
}

# 初始化对话历史
messages = [SYSTEM_PROMPT]

print("欢迎使用 Deepseek 智能助手！\n")
print("功能说明：")
print("1. 直接输入问题进行对话")
print("2. 输入'搜索：关键词'进行网络搜索")
print("3. 输入'清除'清除对话历史")
print("4. 输入'退出'结束对话\n")

while True:
    user_input = input("用户: ").strip()

    # 处理退出命令
    if user_input.lower() in ["退出", "exit"]:
        print("对话已结束，再见！")
        break

    # 处理清除历史命令
    if user_input.lower() in ["清除", "clear"]:
        messages = [SYSTEM_PROMPT]
        print("历史记录已清除！\n")
        continue

    # 处理搜索请求
    if user_input.startswith(("搜索:", "搜索：")):
        search_query = user_input[2:].strip().lstrip(':').lstrip('：').strip()

        if not search_query:
            print("请输入搜索关键词！")
            continue

        print(f"正在搜索: {search_query}")

        # 进行网络搜索
        search_results = search_web(search_query)

        # **检查搜索结果是否为错误信息**
        if isinstance(search_results, str):
            print(f"搜索失败: {search_results}\n")
            continue

        if isinstance(search_results, dict) and 'organic' in search_results:
            combined_info = []
            for result in search_results['organic'][:2]:
                title = result.get('title', '')
                link = result.get('link', '')
                snippet = result.get('snippet', '')
                webpage_content = get_webpage_content(link)
                combined_info.append(f"标题: {title}\n链接: {link}\n摘要: {snippet}\n内容: {webpage_content}\n")

            if combined_info:
                user_input = f"请基于以下搜索结果回答关于'{search_query}'的问题：\n\n" + "\n---\n".join(combined_info)
            else:
                user_input = f"搜索'{search_query}'没有找到相关结果。"
        else:
            print("搜索失败，将直接使用 AI 回答。\n")

    # 添加用户输入到对话历史
    messages.append({"role": "user", "content": user_input})

    # 获取 AI 回复
    assistant_reply = get_ai_response(messages)
    print(f"助手: {assistant_reply}\n")

    # 添加助手回复到对话历史
    messages.append({"role": "assistant", "content": assistant_reply})

欢迎使用 Deepseek 智能助手！

功能说明：
1. 直接输入问题进行对话
2. 输入'搜索：关键词'进行网络搜索
3. 输入'清除'清除对话历史
4. 输入'退出'结束对话

用户: 搜索，：糖的优点
助手: 糖作为一种常见的食品添加剂和调味品，具有以下优点：

1. **提供能量**：糖是碳水化合物的一种，能够快速为人体提供能量，尤其是在需要快速补充体力时，糖是一个有效的选择。

2. **改善口感**：糖可以增加食物的甜味，改善食物的口感，使食物更加美味，尤其是在烘焙食品、饮料和甜点中，糖的作用不可或缺。

3. **促进消化**：适量的糖可以刺激胃酸分泌，帮助消化，尤其是在饭后适量摄入糖分，有助于促进消化系统的运作。

4. **情绪调节**：糖可以刺激大脑释放多巴胺，带来愉悦感，有助于缓解压力和焦虑，提升情绪。

5. **保存食品**：糖具有吸水性，可以抑制微生物的生长，因此在食品加工中，糖常被用作防腐剂，延长食品的保质期。

6. **促进发酵**：在烘焙和酿酒过程中，糖是酵母发酵的重要原料，能够促进发酵过程，产生二氧化碳和酒精。

7. **增加食物的色泽**：在烹饪过程中，糖可以通过焦糖化反应，使食物呈现出诱人的金黄色泽，提升食物的视觉效果。

尽管糖有这些优点，但过量摄入糖分可能会导致肥胖、糖尿病、心血管疾病等健康问题，因此建议适量摄入糖分，保持均衡饮食。

用户: 搜索：糖的缺点
正在搜索: 糖的缺点
助手: API调用出错: Expecting value: line 1 column 1 (char 0)

用户: 退出
对话已结束，再见！


In [ ]:
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import quote
import time
import sys

# 初始化 DeepSeek API 客户端
client = OpenAI(api_key="sk-a7d86afd5c5d46f6bcff42afc55fbc96", base_url="https://api.deepseek.com")

def search_web(query, num_results=3):
    """
    使用 Serper API 进行网络搜索
    """
    try:
        headers = {
            "X-API-KEY": "ad64e51d04b48953f7aa8907d28e05db00a2d89b",
            "Content-Type": "application/json"
        }
        payload = {
            "q": query,
            "num": num_results
        }
        response = requests.post(
            "https://google.serper.dev/search",
            headers=headers,
            json=payload
        )
        return response.json()
    except Exception as e:
        return f"搜索出错: {str(e)}"

def get_webpage_content(url):
    """
    获取网页内容并提取文本
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        for script in soup(['script', 'style']):
            script.decompose()

        text = soup.get_text(separator='\n', strip=True)
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        text = '\n'.join(lines)

        return text[:1000]
    except Exception as e:
        return f"获取网页内容出错: {str(e)}"

def print_streaming(text, delay=0.02):
    """
    实现打字机效果的输出
    """
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    sys.stdout.write('\n\n')
    sys.stdout.flush()

def get_ai_streaming_response(messages):
    """
    获取 AI 流式响应
    """
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            stream=True  # 启用流式输出
        )

        # 用于收集完整响应
        full_response = ""
        print("助手: ", end='', flush=True)

        # 处理流式响应
        for chunk in response:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                sys.stdout.write(content)
                sys.stdout.flush()
                full_response += content
                time.sleep(0.02)  # 控制打字速度

        print("\n")
        return full_response

    except Exception as e:
        error_msg = f"API调用出错: {str(e)}"
        print_streaming(error_msg)
        return error_msg

# 初始化基础系统提示
SYSTEM_PROMPT = {
    "role": "system",
    "content": "You are a helpful assistant. When users specifically request web search with '搜索：', you'll provide information based on search results. Otherwise, you'll respond based on your own knowledge."
}

# 初始化对话历史
messages = [SYSTEM_PROMPT]

print_streaming("欢迎使用 Deepseek 智能助手！\n")
print("功能说明：")
print("1. 直接输入问题进行对话")
print("2. 输入'搜索：关键词'进行网络搜索")
print("3. 输入'清除'清除对话历史")
print("4. 输入'退出'结束对话\n")

while True:
    user_input = input("用户: ").strip()

    # 处理退出命令
    if user_input.lower() in ["退出", "exit"]:
        print_streaming("对话已结束，再见！")
        break

    # 处理清除历史命令
    if user_input.lower() in ["清除", "clear"]:
        messages = [SYSTEM_PROMPT]
        print_streaming("历史记录已清除！")
        continue

    # 处理搜索请求
    if user_input.startswith(("搜索:", "搜索：")):
        search_query = user_input[2:].strip().lstrip(':').lstrip('：').strip()

        if not search_query:
            print_streaming("请输入搜索关键词！")
            continue

        print_streaming(f"正在搜索: {search_query}")

        # 进行网络搜索
        search_results = search_web(search_query)

        if isinstance(search_results, dict) and 'organic' in search_results:
            combined_info = []
            for result in search_results['organic'][:2]:
                title = result.get('title', '')
                link = result.get('link', '')
                snippet = result.get('snippet', '')
                webpage_content = get_webpage_content(link)
                combined_info.append(f"标题: {title}\n链接: {link}\n摘要: {snippet}\n内容: {webpage_content}\n")

            if combined_info:
                user_input = f"请基于以下搜索结果回答关于'{search_query}'的问题：\n\n" + "\n---\n".join(combined_info)
            else:
                user_input = f"搜索'{search_query}'没有找到相关结果。"
        else:
            print_streaming("搜索失败，将直接使用 AI 回答。")

    # 添加用户输入到对话历史
    messages.append({"role": "user", "content": user_input})

    # 获取 AI 流式回复
    assistant_reply = get_ai_streaming_response(messages)

    # 添加助手回复到对话历史
    messages.append({"role": "assistant", "content": assistant_reply})

欢迎使用 Deepseek 智能助手！


功能说明：
1. 直接输入问题进行对话
2. 输入'搜索：关键词'进行网络搜索
3. 输入'清除'清除对话历史
4. 输入'退出'结束对话

用户: 1. "DeepSeek事件导致英伟达股价大幅下跌，进而引发纳斯达克指数下跌。请问纳斯达克指数未来是否有回升的可能性？"  2. "如果特朗普再次当选美国总统，这对道琼斯工业平均指数会产生怎样的影响？"
助手: 1. 关于纳斯达克指数未来是否有回升的可能性，这取决于多种因素，包括全球经济状况、市场情绪、科技行业的表现以及宏观经济政策等。虽然DeepSeek事件可能对英伟达股价产生了短期影响，但长期来看，纳斯达克指数的走势将更多地受到整体经济和行业基本面的驱动。投资者应密切关注市场动态和相关经济指标，以做出更为明智的投资决策。

2. 如果特朗普再次当选美国总统，对道琼斯工业平均指数的影响将取决于他的政策方向和实施情况。特朗普在任期间曾推行减税、放松监管等政策，这些措施在短期内可能提振市场信心和企业盈利，从而对股市产生积极影响。然而，他的贸易政策和外交策略也可能带来不确定性，影响市场情绪。总体而言，特朗普的政策对道琼斯工业平均指数的影响将是多方面的，投资者需要综合考虑各种因素。

用户: 请根据历史表现和经验积累，对长期发展态势进行系统性评估
助手: 对长期发展态势进行系统性评估需要综合考虑历史表现、经济基本面、政策环境、技术进步以及全球市场动态等多个维度。以下是对纳斯达克指数和道琼斯工业平均指数（DJIA）长期发展态势的系统性评估：

---

### **1. 纳斯达克指数的长期发展态势**
纳斯达克指数以科技股为主，其长期表现与科技创新、全球经济数字化趋势密切相关。

#### **历史表现：**
- **长期增长趋势**：自1971年成立以来，纳斯达克指数整体呈现上升趋势，尤其是在过去20年中，受益于互联网、云计算、人工智能和生物技术等领域的快速发展。
- **波动性较高**：由于科技股的高成长性和高估值特性，纳斯达克指数在短期内可能经历较大波动（如2000年互联网泡沫破裂和2022年科技股回调）。
- **韧性较强**：尽管经历多次回调，纳斯达克指数通常在调整后迅速恢复并创下新高，显示出科技行业的长期增长潜力。

#### **长期驱动因素：**
- **科技创新**

KeyboardInterrupt: 

In [ ]:
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
import json
import time
import sys

# 初始化 DeepSeek API 客户端
client = OpenAI(api_key="sk-a7d86afd5c5d46f6bcff42afc55fbc96", base_url="https://api.deepseek.com")

def search_web(query, num_results=3):
    """使用 Serper API 进行网络搜索"""
    try:
        headers = {
            "X-API-KEY": "ad64e51d04b48953f7aa8907d28e05db00a2d89b",
            "Content-Type": "application/json"
        }
        payload = {"q": query, "num": num_results}
        response = requests.post("https://google.serper.dev/search", headers=headers, json=payload)
        return response.json()
    except Exception as e:
        return f"搜索出错: {str(e)}"

def get_webpage_content(url):
    """获取网页内容并提取文本"""
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        for script in soup(['script', 'style']):
            script.decompose()

        text = soup.get_text(separator='\n', strip=True)
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        return '\n'.join(lines)[:1000]

    except Exception as e:
        return f"获取网页内容出错: {str(e)}"

def print_streaming(text, delay=0.01):  # 进一步加速流式输出
    """实现打字机效果的输出"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    sys.stdout.write('\n\n')
    sys.stdout.flush()

def optimize_prompt(prompt):
    """先使用 DeepSeek 处理优化提示词"""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role": "system", "content": "请优化以下提示词，使其更加清晰和专业：" + prompt}],
            stream=False
        )
        return response.choices[0].message.content.strip()  # 仅返回优化后的提示词
    except Exception as e:
        return f"提示词优化失败: {str(e)}"

def get_ai_streaming_response(messages):
    """获取 AI 流式响应，支持长回复的实时显示"""
    try:
        print_streaming("助手正在思考，请稍候...\n")

        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            stream=True  # 启用流式输出
        )

        full_response = ""
        print("助手: ", end='', flush=True)

        for chunk in response:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                sys.stdout.write(content)
                sys.stdout.flush()
                full_response += content
                time.sleep(0.01)  # 进一步减少等待时间

        print("\n")
        return full_response

    except Exception as e:
        error_msg = f"API调用出错: {str(e)}"
        print_streaming(error_msg)
        return error_msg

# 初始化基础系统提示
SYSTEM_PROMPT = {
    "role": "system",
    "content": "You are a helpful assistant. When users specifically request web search with '搜索：', you'll provide information based on search results. Otherwise, you'll respond based on your own knowledge."
}

# 初始化对话历史
messages = [SYSTEM_PROMPT]

print_streaming("欢迎使用 Deepseek 智能助手！\n")
print("功能说明：")
print("1. 直接输入问题进行对话")
print("2. 输入'搜索：关键词'进行网络搜索")
print("3. 输入'清除'清除对话历史")
print("4. 输入'退出'结束对话\n")

while True:
    user_input = input("用户: ").strip()

    if user_input.lower() in ["退出", "exit"]:
        print_streaming("对话已结束，再见！")
        break

    if user_input.lower() in ["清除", "clear"]:
        messages = [SYSTEM_PROMPT]
        print_streaming("历史记录已清除！")
        continue

    if user_input.startswith(("搜索:", "搜索：")):
        search_query = user_input[2:].strip().lstrip(':').lstrip('：').strip()

        if not search_query:
            print_streaming("请输入搜索关键词！")
            continue

        print_streaming(f"正在搜索: {search_query}")

        search_results = search_web(search_query)

        if isinstance(search_results, dict) and 'organic' in search_results:
            combined_info = []
            for result in search_results['organic'][:2]:
                title = result.get('title', '')
                link = result.get('link', '')
                snippet = result.get('snippet', '')
                webpage_content = get_webpage_content(link)
                combined_info.append(f"标题: {title}\n链接: {link}\n摘要: {snippet}\n内容: {webpage_content}\n")

            if combined_info:
                user_input = f"请基于以下搜索结果回答关于'{search_query}'的问题：\n\n" + "\n---\n".join(combined_info)
            else:
                user_input = f"搜索'{search_query}'没有找到相关结果。"
        else:
            print_streaming("搜索失败，将直接使用 AI 回答。")

    # **优化提示词**
    print_streaming("正在优化提示词...\n")
    optimized_prompt = optimize_prompt(user_input)
    print_streaming(f"优化后的提示词: {optimized_prompt}\n")

    # **使用优化后的提示词作为最终提问**
    messages.append({"role": "user", "content": optimized_prompt})

      # **获取 AI 流式回复**
    assistant_reply = get_ai_streaming_response(messages)

    # **添加助手回复到对话历史**
    messages.append({"role": "assistant", "content": assistant_reply})

欢迎使用 Deepseek 智能助手！


功能说明：
1. 直接输入问题进行对话
2. 输入'搜索：关键词'进行网络搜索
3. 输入'清除'清除对话历史
4. 输入'退出'结束对话

用户: 纳斯达克因为deepseek让英伟达暴跌而暴跌，还会涨回来吗？特朗普上台有利于道琼斯指数嘛？
正在优化提示词...


优化后的提示词: 优化后的提示词：

1. "DeepSeek事件导致英伟达股价大幅下跌，进而引发纳斯达克指数下跌。请问纳斯达克指数未来是否有回升的可能性？"

2. "如果特朗普再次当选美国总统，这对道琼斯工业平均指数会产生怎样的影响？"

优化说明：
1. 将"暴跌"改为更专业的"大幅下跌"
2. 明确因果关系，使用"导致"、"引发"等连接词
3. 将"还会涨回来吗"改为更专业的"未来是否有回升的可能性"
4. 将"上台"改为更正式的"当选"
5. 使用完整的指数名称"道琼斯工业平均指数"
6. 使用更专业的提问方式"会产生怎样的影响"
7. 将两个问题分开，使每个问题更加清晰和专注


助手正在思考，请稍候...


助手: 1. **关于纳斯达克指数未来是否有回升的可能性**：  
   纳斯达克指数的未来走势受多种因素影响，包括宏观经济环境、科技行业的表现、美联储货币政策以及全球经济形势等。虽然DeepSeek事件可能对英伟达股价产生短期冲击，进而影响纳斯达克指数，但长期来看，指数的回升可能性取决于整体市场的基本面和投资者信心。如果科技行业继续保持创新和增长，且宏观经济环境稳定，纳斯达克指数仍有可能回升。

2. **如果特朗普再次当选美国总统，对道琼斯工业平均指数的影响**：  
   特朗普的政策倾向（如减税、放松监管、贸易政策等）可能会对市场产生直接影响。如果他再次当选，市场可能会预期类似的财政刺激政策，这可能在短期内提振道琼斯工业平均指数。然而，贸易政策的不确定性（如关税政策）也可能引发市场波动。总体而言，特朗普的当选可能会对道琼斯指数产生复杂的影响，具体表现取决于其政策实施的具体细节和市场对这些政策的反应。

如果需要更详细的分析或实时数据，建议参考专业财经媒体或咨询金融分析师。

用户: 长期情况呢？根据历史情况和经验而定
正在优化提示词...


优化后的提示词: 优化后的提示词：

"请基于历史数据和过往经验，分析并预测长期趋势。"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-99575bea7562>", line 150, in <cell line: 0>
    optimized_prompt = optimize_prompt(user_input)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-2-99575bea7562>", line 54, in optimize_prompt
    response = client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/resources/chat/completions.py", line 859, in create
    return self._post(
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 1283, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=str

TypeError: object of type 'NoneType' has no len()